In [8]:
# pip install easyAI

In [9]:
#pip install opencv-python

In [12]:
#pip install matplotlib

In [1]:
import time
from easyAI import TwoPlayerGame
from easyAI.Player import Human_Player
from easyAI import AI_Player, Negamax

import random
import numpy as np

In [2]:

LOWERBOUND, EXACT, UPPERBOUND = -1, 0, 1
inf = float("infinity")

def negamax2(game, depth, origDepth, scoring, tt=None):

    alphaOrig = -inf
    # Is there a transposition table and is this game in it ?
    lookup = None if (tt is None) else tt.lookup(game)

    if lookup is not None:
        # The game has been visited in the past

        if lookup["depth"] >= depth:
            flag, value = lookup["flag"], lookup["value"]
            if flag == EXACT:
                if depth == origDepth:
                    game.ai_move = lookup["move"]
                return value
            elif flag == LOWERBOUND:
                alphaOrig = max(alphaOrig, value)
            elif flag == UPPERBOUND:
                alphaOrig = min(alphaOrig, value)

            if alphaOrig == inf:
                if depth == origDepth:
                    game.ai_move = lookup["move"]
                return value

    if (depth == 0) or game.is_over():
        return scoring(game) * (1 + 0.001 * depth)

    if lookup is not None:
        # Put the supposedly best move first in the list
        possible_moves = game.possible_moves()
        possible_moves.remove(lookup["move"])
        possible_moves = [lookup["move"]] + possible_moves
    else:
        possible_moves = game.possible_moves()

    state = game
    best_move = possible_moves[0]
    if depth == origDepth:
        state.ai_move = possible_moves[0]

    bestValue = -inf
    unmake_move = hasattr(state, "unmake_move")

    for move in possible_moves:
        if not unmake_move:
            game = state.copy()  # re-initialize move

        game.make_move(move)
        game.switch_player()

        move_alpha = -negamax2(game, depth - 1, origDepth, scoring, tt)

        if unmake_move:
            game.switch_player()
            game.unmake_move(move)

        # bestValue = max( bestValue,  move_alpha )
        if bestValue < move_alpha:
            bestValue = move_alpha
            best_move = move

        if bestValue == inf:
            break

    if tt is not None:
        assert best_move in possible_moves
        tt.store(
            game=state,
            depth=depth,
            value=bestValue,
            move=best_move,
            flag=EXACT,
        )
    return bestValue


class Negamax2:
    def __init__(self, depth, scoring=None, win_score=+inf, tt=None):
        self.scoring = scoring
        self.depth = depth
        self.tt = tt
        self.win_score = win_score

    def __call__(self, game):
        """
        Returns the AI's best move given the current state of the game.
        """

        scoring = (
            self.scoring if self.scoring else (lambda g: g.scoring())
        )  # horrible hack

        self.alpha = negamax2(
            game,
            self.depth,
            self.depth,
            scoring,
            self.tt,
        )
        return game.ai_move


In [18]:
class TicTac(TwoPlayerGame):
    """Plansza:
    1 2 3
    4 5 6
    7 8 9
    """

    def __init__(self, players):   ##inicjalizacja gry
        self.players = players
        self.board = [0 for i in range(9)]
        self.current_player = 1    #zaczyna gracz 1
        self.wins = np.zeros(3) # indeksy: 0 - ilość remisów; 1 - ilość wygranych gracza 1; 2 - ilość wygranych gracza 2
        # self.times = []
        # self.average = 0

    def possible_moves(self):      #zwraca wszystkie możliwe ruchy
        return [i + 1 for i, e in enumerate(self.board) if e == 0]

    def make_move(self, move):  #zmiana w zależności od ruchu
    

        if random.randint(0,99) > 20:
           self.board[int(move) - 1] = self.current_player     #wersja probabilistyczna

        # self.board[int(move) - 1] = self.current_player        #wersja deterministyczna


    def unmake_move(self, move):   #opcjonalnie (speeds up the AI)
        self.board[int(move) - 1] = 0

    def lose(self):              #czy przeciwnik ma 3 w lini
        return any([all([(self.board[c - 1] == self.opponent_index) for c in line])
                for line in [
                    [1, 2, 3],
                    [4, 5, 6],
                    [7, 8, 9],  # poziome
                    [1, 4, 7],
                    [2, 5, 8],
                    [3, 6, 9],  # pionowe
                    [1, 5, 9],
                    [3, 5, 7],]])  # na ukos

    def is_over(self):      #sprawdza czy koniec gry
        return (self.possible_moves() == []) or self.lose()

    def game_result(self): # wynik jednej gry
        if not self.possible_moves(): # brak dostępnych ruchów -> remis
            self.wins[0] += 1
            print("\nIts a tie!")

        elif self.lose(): #  wyświetla numer gracza, który wygrał
            self.wins[self.opponent_index] += 1
            print(f"\nPlayer {self.opponent_index} wins!")
        

    def show(self):         #pokazuje grę
        print( "\n"+ "\n".join([
                    " ".join([[".", "O", "X"][self.board[3 * j + i]] for i in range(3)])
                    for j in range(3)]))

    def scoring(self):     #punkty
        return -100 if self.lose() else 0


In [19]:
class Game:
    def __init__(self, depth1, depth2, nr_games):
        self.overall_wins = np.zeros(3)
        self.depth1 = depth1
        self.depth2 = depth2
        self.nr_games = nr_games
        self.avg_times = []

    def final_results(self): # pokazuje ilości wygranych ze wszystkich gier
        average = sum(self.avg_times) / len(self.avg_times)
        print(f"\nRESULTS"
              f"\nNumber of games: {int(self.nr_games)}"
              f"\nFirst player wins: {int(self.overall_wins[1])}"
              f"\nSecond player wins: {int(self.overall_wins[2])}"
              f"\nOverall ties: {int(self.overall_wins[0])}"
              f"\nAverage time: {average}")

    def play_games(self):
        # ai_algo1 = Negamax(self.depth1)
        # ai_algo2 = Negamax(self.depth2)
        ai_algo1 = Negamax2(self.depth1)
        ai_algo2 = Negamax2(self.depth2)

        for i in range(1, self.nr_games + 1):
            print("\nGame ", i, ": ")
            player1 = AI_Player(ai_algo1)
            player2 = AI_Player(ai_algo2)
            game = TicTac([player1, player2])

            if not i % 2: # zmiana gracza rozpoczynającego grę
                game.switch_player()

            start_time = time.time()
            game.play()
            end_time = time.time()
            execution_time = end_time - start_time
            game.game_result() # uzyskanie wyników gry
            self.overall_wins += game.wins # dodaje wyniki z poszczególnych gier do końcowych winików
            self.avg_times.append(execution_time)

        self.final_results() #łączne wyniki z każdej gry

In [21]:
if __name__ == "__main__":
    new_game = Game(8,8,10)
    new_game.play_games()


Game  1 : 

. . .
. . .
. . .

Move #1: player 1 plays 1 :

O . .
. . .
. . .

Move #2: player 2 plays 2 :

O X .
. . .
. . .

Move #3: player 1 plays 3 :

O X O
. . .
. . .

Move #4: player 2 plays 4 :

O X O
X . .
. . .

Move #5: player 1 plays 5 :

O X O
X O .
. . .

Move #6: player 2 plays 6 :

O X O
X O X
. . .

Move #7: player 1 plays 7 :

O X O
X O X
O . .

Player 1 wins!

Game  2 : 

. . .
. . .
. . .

Move #1: player 2 plays 1 :

. . .
. . .
. . .

Move #2: player 1 plays 1 :

O . .
. . .
. . .

Move #3: player 2 plays 2 :

O X .
. . .
. . .

Move #4: player 1 plays 3 :

O X O
. . .
. . .

Move #5: player 2 plays 4 :

O X O
X . .
. . .

Move #6: player 1 plays 5 :

O X O
X . .
. . .

Move #7: player 2 plays 5 :

O X O
X X .
. . .

Move #8: player 1 plays 6 :

O X O
X X .
. . .

Move #9: player 2 plays 6 :

O X O
X X .
. . .

Move #10: player 1 plays 6 :

O X O
X X O
. . .

Move #11: player 2 plays 7 :

O X O
X X O
X . .

Move #12: player 1 plays 8 :

O X O
X X O
X O .

Move #

**Cel ćwiczenia :**
Celem ćwiczenia jest stworzenie gry Tic-tac-toe (Kółko i krzyżyk) w języku Python.
Probabilistyczny wariant gry ma zakładać, że z 20% prawdopodobieństwem ruch się nie uda - nie zostanie odnotowany na planszy i nastąpi kolej przeciwnika.

**Opis problemu :**
Kółko i krzyżyk to popularna gra, w której dwóch graczy umieszcza na planszy kolejno kółka i krzyżyki w celu ułożenia trzech symboli w rzędzie, kolumnie lub na skos. Pierwszy gracz, który osiągnie ten cel, wygrywa.

**Realizacja rozwiązania :**
Gra napisana została przez nas w języku programowania Python, w programie JupyterLab.
Składa się ona z planszy o wymiarach 3x3, na której gracze mogą wybierać wolne pola, aż jeden z nich ułoży trzy swoje symbole w rzędzie, kolumnie lub na skos. Wtedy następuje zakończenie gry, a program wyświetla informację o zwycięzcy.
Tworząc grę opierałyśmy się na kodzie z GitHuba (https://github.com/Zulko/easyAI/blob/master/easyAI/games/TicTacToe.py) wykorzystującym EasyAI, czyli platformę  służącą do tworzenia sztucznej inteligencji w grach.
Wykorzystany został przez nas algorytm Negamax - odmiana algorytmu MiniMax. Negamax opiera się na właściwości gry o sumie zerowej - wynik na planszy jest odwrotny dla obu graczy. Działa poprzez przeszukiwanie drzewa gry, czyli wszystkich możliwych ruchów, jakie gracze mogą wykonać na planszy. Dla każdego węzła drzewa ocenia wartość tej pozycji dla gracza, którego ruch jest oceniany, a następnie wybiera ruch, który prowadzi do najlepszego wyniku. Algorytm ten obsługuje również przycinanie alfa/beta, ignorując gałęzie, które nie mogą być lepsze (na podstawie tego, co do tej pory widział), co pozwala na przyspieszenie obliczeń.

#### Rezultaty dla algorytmu Negamax z użyciem obcięcia alfa-beta:
**Wersja probabilistycznej:**
Podczas testowania działania wersji probabilistycznej gry dla głębokości algorytmu Negamax równej 1 można było zaobserwować, poszczególne rozgrywki są do siebie bardzo podobne.
Ilość wygranych gier dla każdego z graczy wynosi ok. 50%, a remisy występują bardzo rzadko. Wraz ze wzrostem liczby poszczególnych rozgrywek rośnie liczba remisów, lecz nadal jest ich niewiele, w porównaniu do łącznej liczby gier.
Przykładowy wynik dla 100 rozgrywek:
- wygrane gracza 1: 45
- wygrane gracza 2: 53
- remisy: 2
- średni czas wykonywania ruchu: 0.0029s

Wynik dla kolejnych 100 rozgrywek:
- wygrane gracza 1: 46
- wygrane gracza 2: 50
- remisy: 4
- średni czas wykonywania ruchu: 0.0025s

Wraz ze wzrostem przewidywanych kroków wprzód rozgrywki są bardziej różnorodne i rośnie liczba remisów. Rośnie też czas wykonywania ruchu.

Przykładowy wynik dla 100 rozgrywek oraz liczby kroków równej 5:
- wygrane gracza 1: 46
- wygrane gracza 2: 46
- remisy: 8
- średni czas wykonywania ruchu: 0.0634s

Wynik dla kolejnych 100 rozgrywek:
- wygrane gracza 1: 38
- wygrane gracza 2: 55
- remisy: 7
- średni czas wykonywania ruchu: 0.0679s

Przykładowy wynik dla 100 rozgrywek oraz liczby kroków równej 8:
- wygrane gracza 1: 45
- wygrane gracza 2: 42
- remisy: 13
- średni czas wykonywania ruchu: 0.7777s

Wynik dla kolejnych 100 rozgrywek:
- wygrane gracza 1: 50
- wygrane gracza 2: 38
- remisy: 12
- średni czas wykonywania ruchu: 0.6943s

Fakt, że przy róznych głębokościach algorytmu Negamax gracz z większą głębokością będzie radził sobie lepiej, nie jest zaskakujący.

Przykładowy wynik dla 50 rozgrywek, liczba kroków gracza 1: 1, liczba kroków gracza 2: 8:
- wygrane gracza 1: 14
- wygrane gracza 2: 33
- remisy: 3

Wynik dla kolejnych 50 rozgrywek:
- wygrane gracza 1: 17
- wygrane gracza 2: 32
- remisy: 1

**Wersja deterministyczna:**
Jeśli chodzi o wersję deterministyczną gry, wyniki są dużo mniej zróżnicowane. Jako że nie pojawia się utrudnienie w postaci możliwości niewykonania ruchu, dla glębokości mniejszej niż 5 kroków nie dochodzi do remisów. Tak jak w wersji probabilistycznej, wraz ze wzrostem głębokości algorytmu Negamax rośnie czas wykonywania ruchu.

Przykładowy wynik dla 100 rozgrywek oraz liczby kroków równej 1:
- wygrane gracza 1: 50
- wygrane gracza 2: 50
- remisy: 0
- średni czas wykonywania ruchu: 0.0020s


Przykładowy wynik dla 100 rozgrywek oraz liczby kroków równej 5:
- wygrane gracza 1: 50
- wygrane gracza 2: 50
- remisy: 0
- średni czas wykonywania ruchu: 0.0415s


Po przekroczeniu liczby kroków > 5 gracze radzą sobie "za dobrze" - ciągle remisują.

Przykładowy wynik dla 100 rozgrywek oraz liczby kroków równej 6:
- wygrane gracza 1: 0
- wygrane gracza 2: 0
- remisy: 100
- średni czas wykonywania ruchu: 0.0785s

Przykładowy wynik dla 30 rozgrywek oraz liczby kroków równej 8:
- wygrane gracza 1: 0
- wygrane gracza 2: 0
- remisy: 30
- średni czas wykonywania ruchu: 0.3357s

W przypadku różnych głębokości algorytmu Negamax gracz z większą głębokością przoduje.

Przykładowy wynik dla 30 rozgrywek, liczba kroków gracza 1: 1, liczba kroków gracza 2: 8:
- wygrane gracza 1: 0
- wygrane gracza 2: 30
- remisy: 0

Przykładowy wynik dla 50 rozgrywek, liczba kroków gracza 1: 1, liczba kroków gracza 2: 4:
- wygrane gracza 1: 0
- wygrane gracza 2: 50
- remisy: 0

---
#### Negamax bez obcięcia alfa-beta:

**Deterministyczna wersja gry:**
Użycie algorytmu negamax bez obcięcia alfa-beta znacząco wpływa na długość gry.
Dla 50 rozgrywek z głębokością równą 8 średni czas wynosi 11.01703092098236 sekund.
Przykładowy wynik:
- wygrane gracza 1: 25
- wygrane gracza 2: 25
- remisy: 0

Dla głębokości równej 3 średni czas jest 0.10609419822692871.
- wygrane gracza 1: 25
- wygrane gracza 2: 25
- remisy: 0

**Probabilistyczna wersja gry:**
Przykładowy wynik dla 10 rozgrywek z głębokością równą 8:
- wygrane gracza 1: 3
- wygrane gracza 2: 3
- remisy: 4
- średni czas: 75.56276068687438


Przykładowy wynik dla 50 rozgrywek z głębokością równą 3:
- wygrane gracza 1: 15
- wygrane gracza 2: 12
- remisy: 23
- średni czas: 0.18524165630340575




**Trudności :**
Podczas tworzenia gry nie pojawiło się zbyt wiele trudności. Najwięcej czasu zajęło zapoznanie się z działaniem biblioteki easy AI i algorytmu Negamax. Późniejszym problemem było obliczenie średniego czasu wykonywania ruchu - po pierwszej próbie dostawałyśmy wyniki, które wraz ze wzrostem głębokości byly coraz mniejsze. Widoczne jednak było, że algorytm wykonywany był coraz dłużej. Ostatecznie udało się rozwiązać ten problem i poprawnie zaimplementowałyśmy obliczanie średniego czasu.
Kolejną trudnością było usunięcie obcięcia alfa-beta z algorytmu Negamax. Stworzyłyśmy klasę Negamax2, gdzie zmomdyfikowałyśmy oryginalne funkcje, tak aby uzyskać działający algorytm bez użycia metody alfa-beta.

**Przemyślenia i wnioski :**
Tworzenie gry w kółko i krzyżyk było ciekawym doświadczeniem, które umożliwiło nam poznanie bilbioteki EasyAI. Wymagało od nas zagłębienia się nie tylko w samą bilbiotekę, ale też w teorię gier w dziedzinie sztucznej inteligencji. 
Ostatecznie, udało się stworzyć działającą grę, która pozwala na rozgrywkę pomiędzy dwoma graczami. Przetestowałyśmy jej działanie na różnych ustawieniach i porównałyśmy wyniki.